In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from oso import Oso
from sqlalchemy_oso.hooks import authorized_sessionmaker
from sqlalchemy_oso.auth import register_models
from sqlalchemy_example.models import *

In [2]:
oso = Oso()
register_models(oso, Model)
oso.load_file("sqlalchemy_example/policy.polar")

In [3]:
user = User(username='user')
manager = User(username='manager', manages=[user])
                                                             
public_user_post = Post(contents='public_user_post',
                        access_level='public',
                        created_by=user)
private_user_post = Post(contents='private_user_post',
                        access_level='private',
                        created_by=user)
private_manager_post = Post(contents='private_manager_post',
                            access_level='private',
                            created_by=manager)
public_manager_post = Post(contents='public_manager_post',
                           access_level='public',
                           created_by=manager)


In [4]:
engine = create_engine('sqlite:///:memory:')
Model.metadata.create_all(engine)
session = Session(bind=engine)
session.add_all([user, manager, public_user_post, private_user_post, private_manager_post, public_manager_post])
session.commit()

In [5]:
AuthorizedSession = authorized_sessionmaker(bind=engine,
                                            get_oso=lambda: oso,
                                            get_user=lambda: user,
                                            get_action=lambda: "read")

In [6]:
session = AuthorizedSession()

Now that we've setup some test data, let's use **oso** to authorize `Post`s that `User(username="user")` can see.

In [7]:
posts = session.query(Post).all()

In [8]:
[p.contents for p in posts]

['public_user_post', 'private_user_post', 'public_manager_post']

`user` can see their own public and private posts, and other public posts.

In [9]:
AuthorizedSession = authorized_sessionmaker(bind=engine,
                                            get_oso=lambda: oso,
                                            get_user=lambda: manager,
                                            get_action=lambda: "read")

In [10]:
session = AuthorizedSession()

Now we'll authorize access to `manager`'s `Post`s.

In [11]:
posts = session.query(Post).all()

In [12]:
[p.contents for p in posts]

['public_user_post',
 'private_user_post',
 'private_manager_post',
 'public_manager_post']

We got 4 posts this time, the manager's public and private posts, other user's private posts **and** private posts of users that the `manager` user manages.

In [13]:
manager.manages[0].username

'user'

More complex queries can be authorized as well.

In [14]:
AuthorizedSession = authorized_sessionmaker(bind=engine,
                                            get_oso=lambda: oso,
                                            get_user=lambda: user,
                                            get_action=lambda: "read")
session = AuthorizedSession()

In [15]:
session.query(Post.contents).all()

[('public_user_post'), ('private_user_post'), ('public_manager_post')]

In [16]:
session.query(Post.contents, User.username).all()

[('public_user_post', 'user'),
 ('public_user_post', 'manager'),
 ('private_user_post', 'user'),
 ('private_user_post', 'manager'),
 ('public_manager_post', 'user'),
 ('public_manager_post', 'manager')]

In [17]:
session.query(Post.contents, User.username).join(User).all()

[('public_user_post', 'user'),
 ('private_user_post', 'user'),
 ('public_manager_post', 'manager')]